In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sentence_transformers import SentenceTransformer, util

# Load model and data
model = SentenceTransformer('all-MiniLM-L6-v2')
data = pd.read_csv("/content/drive/MyDrive/MEDICO-PROJECT/Diseases_Symptoms_with_Diet.csv")
hospital_data = pd.read_csv("/content/drive/MyDrive/MEDICO-PROJECT/hostwithpinandloco.csv", encoding='ISO-8859-1', quotechar='"')

symptom_embeddings = model.encode(data['Symptoms'].tolist(), convert_to_tensor=True)

# Core logic
def find_condition(input_symptoms, bp_input=None, age_input=None):
    input_embedding = model.encode(input_symptoms, convert_to_tensor=True)
    cosine_scores = util.pytorch_cos_sim(input_embedding, symptom_embeddings)
    most_similar_idx = cosine_scores[0].argmax().item()

    row = data.iloc[most_similar_idx]
    disease = row['Name']
    treatment = row['Treatments']
    diet = row['Diet']
    relevant_bp = row['Relevant_BP'] == 'Yes'
    relevant_age = row['Relevant_Age'] == 'Yes'

    warning = ""

    # BP handling
    if relevant_bp and bp_input:
        try:
            systolic, diastolic = map(int, bp_input.split("/"))
            if systolic >= 180 or diastolic >= 120 or systolic <= 80 or diastolic <= 50:
                warning = "🚨 <span style='color:red'><b>Bro you are cooked. Please go for an immediate check-up.</b></span>"
            if (systolic >= 140 or diastolic >= 90) and "hypertension" in disease.lower():
                disease += " (Confirmed by high BP)"
            elif (systolic <= 90 or diastolic <= 60) and "hypotension" in disease.lower():
                disease += " (Confirmed by low BP)"
        except:
            pass  # skip BP parsing errors

    # Age handling
    if relevant_age and age_input:
        try:
            age = int(age_input)
            if age < 12 and "chickenpox" in disease.lower():
                disease += " (Common in children)"
            elif age > 60 and any(k in disease.lower() for k in ['alzheimer', 'parkinson']):
                disease += " (Common in elderly)"
        except:
            pass  # skip Age parsing errors

    return disease, treatment, diet, warning

def find_hospital_by_input(user_input):
    if not user_input:
        return "⚠️ Please enter a pincode or location to find nearby hospitals."

    try:
        if user_input.isdigit():
            match = hospital_data[hospital_data['Pincode'] == int(user_input)]
        else:
            match = hospital_data[hospital_data['Location'].str.contains(user_input, case=False, na=False)]

        if not match.empty:
            hospital_name = match.iloc[0]['Hospital Name']
            address = match.iloc[0]['Address']
            return f"🏥 **Hospital:** {hospital_name}\n📍 **Address:** {address}"
        else:
            return "❌ No hospital found for this location/pincode. Try a nearby area."
    except Exception as e:
        return f"🚨 Error: {str(e)}"

# Streamlit UI
st.set_page_config(page_title="AI Symptom Checker + Hospital Finder", page_icon="💊", layout="centered")
st.title("🤖 Medical Assistant Chatbot")
st.markdown("Enter your symptoms, BP (optional), age (optional), and location to get help, treatment advice, and nearby hospital info.")

with st.form(key="symptom_form"):
    symptoms = st.text_area("📝 Describe your symptoms", placeholder="e.g. fever, body pain, chills...")
    bp = st.text_input("🩺 Optional: Enter your BP (e.g. 120/80)", placeholder="Leave blank if unknown")
    age = st.text_input("🎂 Optional: Enter your age", placeholder="e.g. 25")
    location = st.text_input("📍 Need a doctor nearby? Enter area or pincode", placeholder="e.g. Ameerpet or 500038")
    submit = st.form_submit_button("💡 Get Diagnosis")

if submit:
    if not symptoms.strip():
        st.warning("Please enter your symptoms.")
    else:
        disease, treatment, diet, warning = find_condition(symptoms, bp_input=bp, age_input=age)

        st.subheader("🧠 Likely Diagnosis")
        st.success(f"**Disease:** {disease}")

        if warning:
            st.markdown(warning, unsafe_allow_html=True)

        st.subheader("💊 Suggested Treatment")
        st.info(treatment)

        st.subheader("🥗 Diet Tips")
        st.info(diet)

        st.subheader("🏥 Nearby Hospital Info")
        st.write(find_hospital_by_input(location))


Writing app.py


In [3]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 4s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏

In [4]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 35.231.165.113


In [5]:
!streamlit run app.py &>/content/logs.txt &

In [6]:
!npx localtunnel --port 8501/

⠙Usage: lt --port [num] <options>

Options:
  -p, --port                Internal HTTP server port                 [required]
  -h, --host                Upstream server providing forwarding
                                             [default: "https://localtunnel.me"]
  -s, --subdomain           Request this subdomain
  -l, --local-host          Tunnel traffic to this host instead of localhost,
                            override Host header to this host
      --local-https         Tunnel traffic to a local HTTPS server     [boolean]
      --local-cert          Path to certificate PEM file for local HTTPS server
      --local-key           Path to certificate key file for local HTTPS server
      --local-ca            Path to certificate authority file for self-signed
                            certificates
      --allow-invalid-cert  Disable certificate checks for your local HTTPS
                            server (ignore cert/key/ca options)        [boolean]
  -o, --open        